In [33]:
%matplotlib inline
%load_ext memory_profiler
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import bcolz
from tqdm import tqdm,tnrange, tqdm_notebook
tqdm.monitor_interval = 0
from itertools import islice
import matplotlib.pyplot as plt
import numpy as np
import main; reload(main)
from memory_profiler import profile
from ipywidgets import widgets
from IPython.display import display

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [2]:
trainset = torchvision.datasets.ImageFolder('train',
                                           transform=transforms.Compose([
                                               transforms.RandomResizedCrop(224),
                                               #transforms.RandomHorizontalFlip(),
                                               transforms.ToTensor(),
                                           ]))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8,
                                          shuffle=True, num_workers=1)
print(trainset.classes,trainset.imgs[0],len(trainset.imgs))

(['cat', 'dog'], ('train/cat/cat.0.jpg', 0), 23000)


In [32]:
if 'net' in locals():
    del net
net=main.Net(classes=2)
#net=torchvision.models.resnet18()

In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [34]:
import threading

def train(network, loader, breaking):
    with tnrange(2, desc="Training", unit="epoch") as epochbar:
        for epoch in epochbar:  # loop over the dataset multiple times
            with tqdm_notebook(enumerate(loader, 0),
                      total=len(loader.dataset.imgs)/loader.batch_size,
                      desc="Epoch %d" % (epoch + 1),
                      unit="batch",
                      leave=True) as batchbar:
                for i, data in batchbar:
                    if breaking.value:
                        print('Aborted Training')
                        return

                    # get the inputs
                    inputs, labels = data
                    inputs, labels = Variable(inputs), Variable(labels, requires_grad=False)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward + backward + optimize
                    outputs = network(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    
                    # update text
                    batchbar.set_postfix(loss=loss.data[0])

    print('Finished Training')

def runtest():
    breaking=widgets.ToggleButton(description="Break")
    display(breaking)
    bg=threading.Thread(target=train,args=(net,trainloader,breaking,))
    bg.start()

runtest()

ToggleButton(value=False, description=u'Break')

HBox(children=(IntProgress(value=0, description=u'Training', max=2), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Epoch 1', max=2875), HTML(value=u'')))

Aborted Training

